# 📦 **Supply Chain Optimization using SQL**

## 🧾 **Project Overview**

This project explores operational performance and supply chain efficiency using the DataCo Smart Supply Chain dataset.  
The goal is to uncover delivery inefficiencies, supplier risks, product performance and customer order patterns through advance SQL analysis and visualization in Power BI.

**🔧 Tools Used**:  
- Google BigQuery (SQL)
- Power BI
- Jupyter Notebook (for analysis narrative)
- Python (for minor cleaning and documentation)

**🎯 Focus Areas**:  
- Order and delivery performance
- Inventory and product-level profit analysis
- Late delivery risks and shipping modes
- Regional and warehouse performance

## 📁 **Dataset Import**

The dataset was uploaded to Google BigQuery and imported using Character Map V2 to bypass schema errors.  
A cleaned SQL view was created to normalize field names and remove special characters/spaces for easier querying.

## 🧼 **Schema Cleaning**

### 🧹 Column Standardization

The original dataset included over 50+ fields, many of which had inconsistent formatting (e.g., spaces, parentheses).  
To ensure proper querying and compatibility with Power BI, a cleaned view was created with:  
  
- ✅ snake_case formatting
- ✅ grouped fields (customer, order, product, delivery)
- ✅ renamed financial metrics (e.g., `benefit_per_order` ➝ `revenue_per_order`)  
  
SQL View: `orders_cleaned`  
Source: `DataCoSC.orders`  

In [ ]:
CREATE VIEW DataCoSC.orders_cleaned AS
SELECT

  -- 🧍 CUSTOMER
  `Customer Id` AS customer_id,
  `Customer Fname` AS customer_first_name,
  `Customer Lname` AS customer_last_name,
  `Customer Email` AS customer_email,
  `Customer Password` AS customer_password,
  `Customer Segment` AS customer_segment,
  `Customer Street` AS customer_address,
  `Customer Zipcode` AS customer_zipcode,
  `Customer City` AS customer_city,
  `Customer State` AS customer_state,
  `Customer Country` AS customer_country,

  -- 📦 PRODUCT / ORDER ITEMS
  `Product Card Id` AS product_id,
  `Product Name` AS product_name,
  `Product Description` AS product_description,
  `Product Image` AS product_image,
  `Product Price` AS product_price,
  `Product Status` AS product_status,
  `Order Item Id` AS item_id,
  `Order Item Cardprod Id` AS item_product_id,
  `Order Item Quantity` AS item_quantity,
  `Order Item Total` AS item_total,
  `Order Item Profit Ratio` AS item_profit_ratio,
  `Order Item Discount` AS item_discount,

  -- 📑 ORDER
  Type AS order_type,
  `Order Id` AS order_id,
  `Order Customer Id` AS order_customer_id,
  `order date _DateOrders_` AS order_date,
  `Order Status` AS order_status,
  Sales AS order_sales,
  `Order Profit Per Order` AS order_profit_per_order,

  -- 🚚 SHIPPING
  `Days for shipping _real_` AS days_for_shipping,
  `Days for shipment _scheduled_` AS days_for_shipment_scheduled,
  `shipping date _DateOrders_` AS shipping_date,
  `Shipping Mode` AS shipping_mode,
  `Delivery Status` AS delivery_status,
  Late_delivery_risk AS late_delivery,

  -- 🌍 GEOGRAPHY / MARKET
  Market AS market,
  `Order city` AS order_city,
  `Order State` AS order_state,
  `Order Country` AS order_country,
  `Order Region` AS order_region,
  `Order Zipcode` AS order_zipcode,
  Latitude AS latitude,
  Longitude AS longitude,

  -- 🏬 CATEGORY / DEPARTMENT
  `Category Id` AS category_id,
  `Category Name` AS category_name,
  `Department Id` AS department_id,
  `Department Name` AS department,

  -- 💰 DERIVED METRICS
  `Benefit per order`AS revenue_per_order,
  `Sales per customer` AS sales_per_customer

FROM DataCoSC.orders

### 🧩 Schema Planning

#### ❄️ *Schema Summary*

The schema is planned to follow a normalized snwoflake model to improve flexibility and reduce redundancy.  

**Fact Table:**
- `fact_orders`: contains foreign keys and numerical metrics (e.g., sales, quantity, revenue)  

**Dimension Tables:**
- `dim_customer`: customer profile info (name, email, address, segment)
- `dim_product`: product attributes (name, price, category)
- `dim_category`: product categories and departments
- `dim_shipping`: shipping method, scheduled vs actual days, delays
- `dim_order_location`: geographic info for each order (city, state, country, region)
- `dim_date`:supports order/shipping date hierarchies (e.g., year, quarter, month)  
  
Each dimension joins with the fact table via its respective key, e.g., `fact_orders.customer_id = dim_customer.customer_id`.

#### 🪑 *Tables Creation*

Creation of Fact and Dimension Tables was made by the use of Views.

##### 🗂️ Create Fact Table

> This fact table contains transactional-level order data, including item quantities, revenue, profit, discount, shipping info and foreign keys to customer, product and date dimensions. It is the central table used for business metric calculations.

**Key fields:**
- `order_id` **(primary key)**
- `customer_id`, `product_id`, `order_date`, `shipping_date`
- `product_price`, `item_quantity`, `item_discount`, `item_total`, `item_profit_ratio`
- `order_profit_per_order`, `revenue_per_order`, `sales_per_customer`
- `shipping_mode`, `days_for_shipping`, `days_for_shipment_scheduled`, `late_delivery`, `delivery_status`

In [ ]:
CREATE VIEW DataCoSC.fact_orders AS
SELECT
  order_id,
  order_customer_id AS customer_id,
  item_product_id AS product_id,
  FORMAT_DATE('%d-%m-%Y', DATE(order_date)) AS order_date,
  FORMAT_DATE('%d-%m-%Y', DATE(shipping_date)) AS shipping_date,
  shipping_mode,
  product_price,
  item_quantity,
  item_discount,
  item_total,
  item_profit_ratio,
  order_profit_per_order AS item_profit,
  days_for_shipping,
  days_for_shipment_scheduled,
  revenue_per_order,
  sales_per_customer,
  late_delivery,
  delivery_status
FROM DataCoSC.orders_cleaned

##### 🗂️ Create Dimension Table - Customer

> This dimension table contains customer demographic, geographic and segmentation data. It is grouped by `customer_id` to avoid duplicates and supports customer_level analysis.

**Key fields:**
- `customer_id` **(primary key)**
- `first_name`, `last_name`, `email`, `segment`
- `address`, `city`, `state`, `country`, `zipcode`

In [ ]:
CREATE VIEW DataCoSC.dim_customer AS
SELECT
  customer_id,
  customer_first_name AS first_name,
  customer_last_name AS last_name,
  customer_email AS email,
  customer_segment AS segment,
  customer_address AS address,
  customer_city AS city,
  customer_state AS state,
  customer_country AS country,
  customer_zipcode AS zipcode
FROM DataCoSC.orders_cleaned
GROUP BY
  customer_id,
  customer_first_name,
  customer_last_name,
  customer_email,
  customer_segment,
  customer_address,
  customer_city,
  customer_state,
  customer_country,
  customer_zipcode

##### 🗂️ Create Dimension Table - Product

> This table includes descriptive attributes of each product and links to the category dimension. It supports product-level breakdowns and profitability tracking.

**Key fields:**
- `product_id` **(primary key)**
- `product_name`, `product_description`, `product_image`, `product_price`, `product_status`
- `category_id` **(foreign key to `dim_category`)**

In [ ]:
CREATE VIEW DataCoSC.dim_product AS
SELECT
  product_id,
  product_name,
  product_description,
  product_image,
  product_price,
  product_status,
  category_id
FROM DataCoSC.orders_cleaned
GROUP BY
  product_id,
  product_name,
  product_description,
  product_image,
  product_price,
  product_status,
  category_id

##### 🗂️ Create Dimension Table - Category

> Grouped by `category_id`, this table defines product categories and the departments they belong to. Used for hierarchical product analysis.

**Key fields:**
- `category_id` **(primary key)**
- `category`, `department`, `department_id`

In [ ]:
CREATE VIEW DataCoSC.dim_category AS
SELECT
  category_id,
  category_name AS category,
  department_id,
  department
FROM DataCoSC.orders_cleaned
GROUP BY
  category_id,
  category,
  department_id,
  department

##### 🗂️ Create Dimension Table - Order Location

> This table maps each order to its location of origin, including city, region and country. It enables regional and geographic sales analysis.

**Key fields:**
- `order_id` **(primary key)**
- `order_city`, `order_state`, `order_region`, `order_country`
- `order_zipcode`, `market`

In [ ]:
CREATE VIEW DataCoSC.dim_order_location AS
SELECT
  order_id,
  order_city,
  order_state,
  order_region,
  order_country,
  order_zipcode,
  market
FROM DataCoSC.orders_cleaned
GROUP BY
  order_id,
  order_city,
  order_state,
  order_region,
  order_country,
  order_zipcode,
  market

##### 🗂️ Create Dimension Table - Shipping

> This shipping dimension includes delivery mode, schedule adherence and late delivery flags. Grouped by `order_id`, it is used to evaluate shipping efficiency and logistics performance.

**Key fields:**
- `order_id` **(primary key)**
- `shipping_mode`, `days_for_shipping`, `days_for_shipment_scheduled`
- `late_delivery`, `delivery_status`

In [ ]:
CREATE VIEW DataCoSC.dim_shipping AS
SELECT
  order_id,
  shipping_mode,
  days_for_shipping,
  days_for_shipment_scheduled,
  late_delivery,
  delivery_status
FROM DataCoSC.orders_cleaned
GROUP BY
  order_id,
  shipping_mode,
  days_for_shipping,
  days_for_shipment_scheduled,
  late_delivery,
  delivery_status

##### 🗂️ Create Dimension Table - Date

> This is a calendar dimension used to support flexible time-based analysis. It was generated from the order and shipping ranges in the dataset and includes various date hierarchies.

**Key fields:**
- `date` **(primary key)**
- `year`, `quarter`, `month`, `week`, `day_of_year`
- `month_name`, `weekday`, `weekday_name`
- `year_month`, `full_date`

In [ ]:
CREATE VIEW DataCoSC.dim_date AS
WITH date_bounds AS (
  SELECT
    DATE(MIN(order_date)) AS start_date,
    DATE(MAX(order_date)) AS end_date
  FROM DataCoSC.orders_cleaned
),
date_range AS (
  SELECT
    DATE_ADD(start_date, INTERVAL day_num DAY) AS date
  FROM date_bounds,
      UNNEST(GENERATE_ARRAY(0, DATE_DIFF(end_date, start_date, DAY))) AS day_num
)
SELECT
  FORMAT_DATE('%d-%m-%Y', date) AS date,
  EXTRACT(YEAR FROM date) AS year,
  EXTRACT(QUARTER FROM date) AS quarter,
  EXTRACT(MONTH FROM date) AS month,
  FORMAT_DATE('%B', date) AS month_name,
  EXTRACT(WEEK FROM date) AS week,
  EXTRACT(DAYOFWEEK FROM date) AS weekday,
  FORMAT_DATE('%A', date) AS weekday_name,
  EXTRACT(DAYOFYEAR FROM date) AS day_of_year,
  FORMAT_DATE('%Y-%m', date) AS year_month,
  FORMAT_DATE('%Y-%m-%d', date) AS full_date
FROM date_range

### 🧱 Data Warehouse Schema

This project uses a Snowflake Schema to model the cleaned supply chain dataset.  
The design separates dimensional data into multiple related lookup tables for better normalization, flexibility and scalability.  

The central `fact_orders` tables captures transactional-level order data, while dimensional tables describe products, customers, dates, shipping details and order locations.

#### 🖼️ *Schema Design*

The data warehouse schema for this project follows a **snowflake design**, optimizing dimensions like product, customer, shipping and date to support advanced analysis in a normalized structure.

📁 **Resources**:  
- 🔗 [Schema Diagram (Interactive - dbdiagram.io)](https://dbdiagram.io/d/DataCo-Supply-Chain-Schema-68308545b9f7446da3e05968)  
      Visual schema showing tables and relationships crated using [dbdiagram.io](https://dbdiagram.io/).  
- 📄 [Schema Documentation (DBDocs)](https://dbdocs.io/bsilva1297/DataCo-Supply-Chain-Schema)  
      Auto-generated entity documentation powered by [DBDocs.io](https://dbdocs.io/), based on the schema.  
- 📥 [Download PDF Schema](https://github.com/brunopata/DataCo-Supply-Chain/blob/main/schema/DataCo%20Supply%20Chain%20Tables%20Schema.pdf)  
      Static version of the schema for offline reference or sharing.  

📌 **Tools Used**:  
- `dbdiagram.io` - Schema modeling and ERD generation
- `DBML` - Lightweight schema markup
- `DBDocs` - Auto-published database documentation

#### 👨‍💻 *Schema Modeling Tool*

The data warehouse schema was designed using [dbdiagram.io](https://dbdiagram.io/), a tool for visualizing and generating entity-relationship diagrams (ERDs) using code.  

Below is the DBML (Database Markup Language) code used to create the schema:

In [ ]:
Table fact_orders {
  order_id integer [primary key]
  customer_id integer
  product_id integer
  order_date date
  shipping_date date
  shipping_mode string
  product_price float
  item_total float
  item_quantity integer
  item_discount float
  item_profit_ratio float
  order_profit_per_order float
  revenue_per_order float
  days_for_shipping integer
  days_for_shipment_scheduled integer
  late_delivery boolean
  delivery_status string
  sales_per_customer float
}

Table dim_customer {
  customer_id integer [primary key]
  first_name string
  last_name string
  email string
  segment string
  address string
  zipcode string
  city string
  state string
  country string
}

Table dim_product {
  product_id integer [primary key]
  product_name string
  product_description string
  product_image string
  product_price float
  product_status integer
  category_id integer
}

Table dim_category {
  category_id integer [primary key]
  category string
  department_id integer
  department string
}

Table dim_shipping {
  order_id integer [primary key]
  shipping_mode string
  days_for_shipping integer
  days_for_shipment_scheduled integer
  late_delivery boolean
  delivery_status string
}

Table dim_order_location {
  order_id integer [primary key]
  order_city string
  order_state string
  order_region string
  order_country string
  order_zipcode integer
  market string
}

Table dim_date {
  date date [primary key]
  year integer
  quarter integer
  month integer
  month_name string
  week integer
  weekday integer
  weekday_name string
  day_of_year integer
  year_month string
  full_date string
}

Ref fact_orders_dim_date: fact_orders.order_date > dim_date.date // many-to-one

Ref fact_orders_dim_date: fact_orders.shipping_date > dim_date.date // many-to-one

Ref fact_orders_dim_customer: fact_orders.customer_id > dim_customer.customer_id // many-to-one

Ref fact_orders_dim_product: fact_orders.product_id > dim_product.product_id // many-to-one

Ref dim_product_dim_category: dim_product.category_id > dim_category.category_id // many-to-one

Ref: fact_orders.order_id < dim_shipping.order_id

Ref: fact_orders.order_id < dim_order_location.order_id

## 📊 **Early Data Exploration & Descriptive Analysis**

Before diving into in-depth analysis or business intelligence modelling, this section performs an early-stage exploration of the cleaned dataset.  

The objective of this exploration is to:  

- Validate the quality and completeness of the data
- Understand the distribution of key numerical metrics (e.g., quantity, revenue, profit)
- Identify missing values or potential outliers
- Get a sense of how customers, products, categories and countries are represented

This step ensures data readiness and helps guide decisions on whichs KPIs, dimensions or trends are worth pursuing further in later stages of the analysis.

### 1️⃣ Row & Column Summary

For exploration on the total number of rows and columns of data across the dataset.

In [ ]:
# Total Number of Rows

SELECT COUNT(*) AS total_rows
FROM `DataCoSC.orders_cleaned`

In [ ]:
# Total Number of Columns

SELECT COUNT(*) AS total_columns
FROM `DataCoSC.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'orders_cleaned'

📄 The dataset consists of **180519 rows** and **50 columns**, covering customer, order, product and shipping dimensions.

### 2️⃣ Null & Missing Value Analysis

For critical fields like `order_date`, `customer_id`, `product_id` and `shipping_date`.

In [ ]:
# Total Number of Missing Values across Critical Fields

SELECT
    COUNTIF(order_date IS NULL) AS null_order_date,
    COUNTIF(customer_id IS NULL) AS null_customer_id,
    COUNTIF(product_id IS NULL) AS null_product_id,
    COUNTIF(shipping_date IS NULL) AS null_shipping_date
FROM `DataCoSC.orders_cleaned`

📄 The dataset appears to be properly cleaned, with **no nulls or missing values** across critical fields such as `order_date`, `customer_id`, `product_id` and `shipping_date`.

### 3️⃣ Basic Descriptive Stats

For basic information on finances, customers, regions and sold products.

In [ ]:
# Order-Level KPIs

SELECT
    ROUND(MIN(order_total), 2) AS min_order_value,
    ROUND(MAX(order_total), 2) AS max_order_value,
    ROUND(AVG(order_total), 2) AS avg_order_value,
    ROUND(SUM(order_total), 2) AS total_revenue,

    ROUND(MIN(item_quantity), 2) AS min_item_quantity_per_row,
    ROUND(MAX(item_quantity), 2) AS max_item_quantity_per_row,
    ROUND(AVG(item_quantity), 2) AS avg_item_quantity_per_row,
    SUM(item_quantity) AS total_item_quantity,

    ROUND(SUM(item_discount), 2) AS total_discount,
    ROUND(SUM(order_profit), 2) AS total_profit,

    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(DISTINCT customer_id) AS total_customers,
    COUNT(DISTINCT order_country) AS total_countries
FROM (
    SELECT
        order_id,
        customer_id,
        order_country,
        SUM(item_total) AS order_total,
        SUM(item_quantity) AS item_quantity,
        MAX(order_profit_per_order) AS order_profit,
        SUM(item_discount) AS item_discount
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        order_id,
        customer_id,
        order_country
)

In [ ]:
# Product-Level Stats

SELECT
    COUNT(DISTINCT product_id) AS total_unique_products,
    COUNT(DISTINCT category_id) AS total_categories,
    COUNT(DISTINCT department_id) AS total_departments
FROM `DataCoSC.orders_cleaned`

📄 **Basic Descriptive Stats Insights**:  
- ✅ **Average Order Quantity**: The average order includes `5.84` items, with a minimum of `1` and a maximum of `24`, suggesting a typical basket size and occasional bulk orders.  
- ✅ **Order Value Range**: Total order values `33054402.38`, range from as low as `7.49` to as high as `2768.41`, with an average around `502.71`. This indicates a variability in customer purchase behavior (likely due to product type or promotions).  
- ✅ **Total Profit Generated**: Across all orders, the dataset captures a total profit of `4236362.68`, providing a reference point for margin-focused analysis.
- ✅ **Total Orders and Items Sold**: A total of `65752` unique orders were processed, comprising `384079` individual items. This indicates bundled purchasing or multi-product orders.
- ✅ **Discounts Accumulated**: Customers redeemed `3730378.4` in total discounts, representing a significant cost center for the business. This signals aggressive pricing strategies or maybe loyalty-driven promotions. It warrants further analysis of discount effectiveness vs. profit impact.
- ✅ **Customer & Product Reach**:  
  - The dataset contains `20652` unique customers and `118` unique products, `51` different categories and `11` total departments, supporting customer segmentation and product performance analysis.
  - Orders were placed across `164` different countries, making geographic breakdowns meaningful.

These stats provide a foundation for deeper investigation into customer behavior, product profitability and market performance.

### 4️⃣ Top Products / Categories / Customers / Country

For top selling categories and products, countries with most orders and best customers.

#### *Top Products*

In [ ]:
# Top 10 Products

SELECT
    product_name,
    COUNT(*) AS orders
FROM `DataCoSC.orders_cleaned`
GROUP BY product_name
ORDER BY orders DESC
LIMIT 10

📄 These product generate the highest number of sales, making them strong candidates for promotions, inventory priorization and forecasting.
  
  1. **Perfect Fitness Perfect Rip Deck**
  2. **Nike Men's CJ Elite 2 TD Football Cleat**
  3. **Nike Men's Dri-FIT Victory Golf Polo**
  4. **O'Brien Men's Neoprene Life Vest**
  5. **Field & Stream Sportsman 16 Gun Fire Safe**
  6. **Pelican Sunstream 100 Kayak**
  7. **Diamondback Women's Serene Classic Comfort Bi**
  8. **Nike Men's Free 5.0+ Running Shoe**
  9. **Under Armour Girls' Toddler Spine Surge Runni**
  10. **Fighting video games**

#### *Top Categories*

In [ ]:
# Top 5 Categories

SELECT
    category_name,
    COUNT(*) AS orders
FROM `DataCoSC.orders_cleaned`
GROUP BY category_name
ORDER BY orders DESC
LIMIT 5

📄 Product line strategies can be tailored based on revenue vs. volume insights.  

  1. **Cleats**
  2. **Men's Footwear**
  3. **Women's Apparel**
  4. **Indoor/Outdoor Games**
  5. **Fishing**

#### *Top Customers*

In [ ]:
# Top 10 Customers

SELECT
    customer_id,
    customer_name,
    customer_first_name,
    customer_last_name,
    COUNT(*) AS orders
FROM (
    SELECT
        order_id,
        customer_id,
        CONCAT(customer_first_name, ' ', customer_last_name) AS customer_name,
        customer_first_name,
        customer_last_name
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        order_id,
        customer_id,
        customer_first_name,
        customer_last_name
)
GROUP BY customer_id, customer_name, customer_first_name, customer_last_name
ORDER BY orders DESC
LIMIT 10

📄 Top customers are all within the same range of orders, these values of sales show that loyalty programs or tailored offers could be ideal.  

  1. **15 orders**
  2. **14 orders**
  3. **14 orders**
  4. **14 orders**
  5. **14 orders**
  6. **14 orders**
  7. **13 orders**
  8. **13 orders**
  9. **13 orders**
  10. **13 orders**

#### *Top Countries*

In [ ]:
# Top 10 Countries

SELECT
    order_country,
    COUNT(*) AS orders
FROM (
    SELECT
        order_id,
        order_country
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        order_id,
        order_country
)
GROUP BY order_country
ORDER BY orders DESC
LIMIT 10

📄 These regions may represent the company's core market and provide opportunities for regional optimization and expansion.  

  1. **United States**
  2. **France**
  3. **Mexico**
  4. **Australia**
  5. **Germany**
  6. **United Kingdom**
  7. **Brazil**
  8. **China**
  9. **India**
  10. **Indonesia**

## 📈 **Advanced KPIs for Supply Chain Analysis**

Each following block includes:
- ✅ **KPI Name**
- 🎯 **Business Purpose**
- 📊 **SQL Code**
- 💡 **What to Look for in the Result?**
- 🧠 **Insight Summary**

### 1️⃣ Total Revenue and Profit

🎯 **Why it matters:**  
This is your fundamental business performance metric. It helps evaluate **overall company health**, margins and profitability at macro level.

In [ ]:
# 📊 Total Revenue and Profit

SELECT
    COUNT(DISTINCT order_id) AS total_orders,
    ROUND(SUM(order_total), 2) AS total_revenue,
    ROUND(SUM(order_profit), 2) AS total_profit,
    (ROUND(SUM(order_profit) / SUM(order_total), 4)) * 100 AS profit_margin
FROM (
    SELECT
        order_id,
        SUM(item_total) AS order_total,
        MAX(order_profit_per_order) AS order_profit
    FROM `DataCoSC.orders_cleaned`
    GROUP BY order_id
)

💡 **What to Look for in the Result:**
- Revenue = Total Earnings from Sales
- Profit = Remaining Income after Cost
- Margin (%) = How much of Sales becomes Profit

🧠 **Insight:**
A **profit margin between 10-20%** is considered healthy for most product-based businesses. While the current margin falls within this range, it leans toward the **lower end**, suggesting potential leakage. It is recommended to investigate areas such as **excessive discounting**, **low product markup** or **high logistics costs** that may be reducing overall profitability.

### 2️⃣ Average Days to Ship *(Real vs Scheduled)*

🎯 **Why it matters:**  
Tracks fulfillment efficiency. Comparing scheduled vs actual shipment reveals potential inefficiencies or overpromising.

In [ ]:
# 🚚 Shipping Timelines

SELECT
    ROUND(AVG(days_for_shipping), 2) AS avg_actual_days_to_ship,
    ROUND(AVG(days_for_shipment_scheduled), 2) AS avg_scheduled_days_to_ship,
    ROUND(AVG(days_for_shipping - days_for_shipment_scheduled), 2) AS avg_delay_days
FROM `DataCoSC.orders_cleaned`

💡 **What to Look for in the Result:**
- Actual > Scheduled → Delays
- Actual ≈ Scheduled → Efficient Operations
- Actual >> Scheduled → Promises May be Unrealistic

🧠 **Insight:**  
If **average shipping delays** consistently exceed **1-2 days**, it may point to issues in **fulfillment workflows**, **inventory readiness** or **carrier coordination**. However, with an average delay of approximately **0.5 days**, current operations appear to be **efficient and well-aligned with scheduled expectations** — supporting strong **SLA compliance** and contributing positively to **customer satisfaction**.

### 3️⃣ Late Delivery Rate

🎯 **Why it matters:**  
Shows how often shipments fail to arrive on time. Late delivery can damage brand reputation and signal breakdowns in supply chain or logistics coordination.

In [ ]:
# 📦 Late Delivery Rate

SELECT
    COUNTIF(late_delivery = 1) AS late_deliveries,
    COUNT(order_id) AS total_orders,
    ROUND(COUNTIF(late_delivery = 1) / COUNT(order_id), 4) * 100 AS late_delivery_rate
FROM (
    SELECT
        order_id,
        late_delivery
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        order_id,
        late_delivery
)

💡 **What to Look for in the Result:**  
- A **late delivery rate below 5-10%** is acceptable in most industries.
- Above 10% → operational issues in shipping method, partner or fulfillment.

🧠 **Insight:**  
A **late delivery rate above 10%** is generally considered excessive — in this case, the rate is approximately **54.8%**, indicating significant issues in **last-mile delivery**, **shipping mode realiability** or **logistics execution**. While the **average delay is only about 0.5 days**, the high rate suggests that **scheduled delivery windows may be too aggresive or unrealistic**. Addressing delivery expectations and optimizing logistics is essential to improve **customer satisfaction** and uphold **service-level commitments**.

### 4️⃣ Revenue per Customer (RFM Foundation)

🎯 **Why it matters:**  
Forms the base for RFM (Recency, Frequency, Monetary) analysis. Identifies top-spending customers for loyalty, retention and segmentation strategies.

In [ ]:
# 👥 Revenue per Customer

SELECT
    customer_id,
    CONCAT(first_name, ' ', last_name) AS customer_name,
    ROUND(SUM(order_total), 2) AS total_spent,
    COUNT(order_id) AS total_orders,
    ROUND(SUM(order_total) / COUNT(order_id), 2) AS avg_order_value
FROM (
    SELECT
        order_id,
        customer_id,
        SUM(item_total) AS order_total
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        order_id,
        customer_id
) AS orders
JOIN `DataCoSC.dim_customer` USING (customer_id)
GROUP BY
    customer_id,
    customer_name
ORDER BY
    total_spent DESC
LIMIT 10

💡 **What to Look for in the Result:**  
- High Total Spend = Most Valuable Customers
- High Frequency = Loyal Buyers
- High AOV = Customers Who Buy Big per Order

🧠 **Insight:**  
Identifying customers with **high total spend** and **frequent purchases** enables targeted **retention strategies** such as **loyalty programs**, **exclusive offers** or **priority support**. Among top customers, **Average Order Value (AOV)** ranges between **527** and **765**, but it's important to note that **AOV alone doesn't reflect customer loyalty**. For instance, some customers place **a single high-value order (e.g., 1500)** — indicating opportunistic buyers rather than consistent revenue drivers. Segmentation should combine **spend** and **frequency** for a more accurate picture of customer value.

### 5️⃣ Shipping Mode Performance

🎯 **Why it matters:**  
Helps you optimize logistics by identifying the most efficient and reliable shipping methods based on time and delay rates.

In [ ]:
# 🚛 Shipping Mode Performance

SELECT
    shipping_mode,
    COUNT(order_id) AS total_orders,
    ROUND(AVG(days_for_shipping), 2) AS avg_shipping_time,
    ROUND(AVG(late_delivery), 4) * 100 AS late_delivery_rate
FROM (
    SELECT
        shipping_mode,
        order_id,
        days_for_shipping,
        late_delivery
    FROM `DataCoSC.orders_cleaned`
    GROUP BY
        shipping_mode,
        order_id,
        days_for_shipping,
        late_delivery
)
GROUP BY shipping_mode
ORDER BY total_orders DESC

💡 **What to Look for in the Result:**  
- Is faster always better?
- Are some Shipping Modes late more often?
- Which is used most frequently?

🧠 **Insight:**  
Some shipping modes may offer **speed** at the expense of **reliability**. For example, methods labeled **"Same Day"** may still experience frequent delays, which can undermine **customer trust** and **brand perception**. This analysis should be used to balance **cost**, **timeliness** and **delivery consistency** — guiding decisions to **renegotiate with underperforming carriers** or to **adjust delivery promises** based on historical performance by **shipping mode**.

### 6️⃣ Average Items per Order

🎯 **Why it matters:**  
This KPI reflects **average cart size** and helps understand **buying behavior**. A high number can indicate successful bundling strategies or product pairings.

In [ ]:
# 🛒 Average Items per Order

SELECT
    ROUND(AVG(items_per_order), 2) AS avg_items_per_order
FROM (
    SELECT
        order_id,
        SUM(item_quantity) AS items_per_order
    FROM `DataCoSC.orders_cleaned`
    GROUP BY order_id
)

💡 **What to Look for in the Result:**  
- What is the average number of items purchased in a single transaction?
- Are customers buying multiple products per order?
- Can bundling or cross-selling strategies be improved?

🧠 **Insight:**  
An **average cart size of 1.5-3.0** is typical in many e-commerce or retail operations. Here, the **average items per order** is **5.84** which is notably higher, suggesting effective **bundled promotions** or **multi-item discounts**, potential success in **volume-based strategies** or strong **customer buying intent**. This supports continuing or expanding cross-sell strategies and could also influence **inventory forecasting** and **warehouse optimization**.

### 7️⃣ Revenue by Region / Country

🎯 **Why it matters:**  
Analyzing revenue by geography allows you to detect **top-performing markets** and **regions with expansion potential** or **logistics inefficiencies**.

In [ ]:
# 🌎 Revenue by Region / Country

SELECT
    order_country,
    order_region,
    ROUND(SUM(item_total), 2) AS total_revenue,
    ROUND(SUM(order_profit_per_order), 2) AS total_profit,
    ROUND((SUM(order_profit_per_order) / SUM(item_total)) * 100, 2) AS profit_margin,
    COUNT(DISTINCT order_id) AS total_orders,
    ROUND(SUM(SUM(item_total)) OVER (PARTITION BY order_country), 2) AS country_total_revenue
FROM `DataCoSC.orders_cleaned`
GROUP BY
    order_country,
    order_region
ORDER BY
    country_total_revenue DESC,
    total_revenue DESC

💡 **What to Look for in the Result:**  
- What countries drive the most revenue?
- Are high-revenue regions also the most profitable?
- Any countries with high order counts but low profit margins?

🧠 **Insight:**  
There is no significant **regional imbalance** in performance — we don't observe countries with with strong sales but **disproportionately low profitability**. This suggests that **discounting strategies**,**logistics** and **cost structures** are relatively consistent across markets.  
Most of the sales are **concentrated in North America**, while **Europe**, **Asia** and **Australia** contribute moderate revenue. These regions could present **growth opportunities** with focused market expansion or pricing strategies.

## 📝 **Reporting Summary**

This supply chain analytics project focused on uncovering **operational**, **customer** and **logistics insights** using SQL on a structured snowflake schema in Google BigQuery.  
  
A clean schema was developed from the raw dataset, **enabling granular exploration of metrics** such as **shipping efficiency**, **revenue contribution**, **product performance** and **customer value**.

### 🔑 Key Metrics & KPIs

- **Total Orders:** `65,752 orders`
- **Total Revenue:** `$33,054,402.38`
- **Average Cart Size:** `5.84 items`
- **Average Shipping Delay:** `0.57 days`
- **Late Delivery Rate:** `54.82%`
- **Profit Margin:** `12.82%`
- **Top Customers:** `Contribute up to $9,436.61 individually`

### 📌 Key Business Insights

- **Shipping Performance**: While the **averate delay is low**, the **late delivery rate remains high**, suggesting issues like **overly aggressive scheduling** or **last-mile inefficiencies**.
- **Profit Margins**: Slightly higher than typical retail benchmarks, but potential **margin erosion** could stem from **heavy discounting** or**logistics costs**.
- **Cart Size**: Above industry average — indicating strong **bundled purchasing behavior** or successful **volume-based incentives**.
- **Regional Sales**: **North America leads in revenue**, but **Europe**, **Asia** and **Australia** show **growth potential** and merit targeted **market expansion** strategies.
- **High-Value Customers**: A small group of customers contributes a **significant share of total revenue**, supporting the case for **segmentation**, **VIP loyalty programs** and **retention marketing**.

### ✅ Recommendations

- **Reevaluate SLA Commitments**: Recalibrate expected delivery windows and prioritize high-delay shipping modes for process optimization.
- **Margin Optimization**: Analyze discount thresholds, revisit underperforming product pricing and audit regions with low profitability.
- **Expand in Emerging Markets**: Explore targeted campaigns in Europe and Asia based on stable order volumes and moderate profitability.
- **Boost Cross-Sell Tactics**: Leverage the large cart sizes to introduce smart product recommendations, bundles and category-targeted promotions.
- **Segment High-Value Customers**: Create customer tiers to apply differentiated engagement — VIP offers, exclusive deals or early-access launches.

### 📁 What Was Built

- **Normalized Schema** (fact + 5 dimension tables)
- **7 Advanced KPIs** including **profit margin**, **AOV**, **delay rates**, etc.
- **Insight-Driven Analysis** tied to business recommendations
- Documented in a clear, organized Jupyter Notebook with SQL-first logic

### 🚀 Next Steps

- Add **Power BI dashboards** to visualize performance trends
- Enhance CLTV model with time-based analysis (Recency, Frequency, Monetary)
- Integrate cancellation data for net revenue analysis
- Segment customers by region and product preferences for advanced targeting